In [230]:
import helper, consts, test_helper
import importlib
importlib.reload(consts)
importlib.reload(test_helper)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, scipy, requests

In [231]:
# USER
USER = "HOANG"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

# REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
# REGRESSION_TYPES

In [232]:
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0"], 
                            ["relvol_lst15_0", "rrirpnxm_lst15_0"],
                            ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
                            ["relvol_toxhr_0", "rrirpnxm_toxhr_0"]]

FEATURE_COL_NAMES = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0", "rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]

In [233]:
importlib.reload(test_helper)
data = test_helper.Data(FILE_PATH)
train_df = data.update_and_get_train_df(FILE_PATH, "20150701", 31)

Filtered File Dates: ['data.20150102_1200', 'data.20150105_1200', 'data.20150106_1200', 'data.20150107_1200', 'data.20150108_1200', 'data.20150109_1200', 'data.20150112_1200', 'data.20150113_1200', 'data.20150114_1200', 'data.20150115_1200', 'data.20150116_1200', 'data.20150120_1200', 'data.20150121_1200', 'data.20150122_1200', 'data.20150123_1200', 'data.20150126_1200', 'data.20150127_1200', 'data.20150128_1200', 'data.20150129_1200', 'data.20150130_1200', 'data.20150202_1200', 'data.20150203_1200', 'data.20150204_1200', 'data.20150205_1200', 'data.20150206_1200', 'data.20150209_1200', 'data.20150210_1200', 'data.20150211_1200', 'data.20150212_1200', 'data.20150213_1200', 'data.20150217_1200', 'data.20150218_1200', 'data.20150219_1200', 'data.20150220_1200', 'data.20150223_1200', 'data.20150224_1200', 'data.20150225_1200', 'data.20150226_1200', 'data.20150227_1200', 'data.20150302_1200', 'data.20150303_1200', 'data.20150304_1200', 'data.20150305_1200', 'data.20150306_1200', 'data.2015

In [234]:
importlib.reload(test_helper)
test_dfs = data.get_df_between_date(data_path = FILE_PATH, start_date="20150701", end_date="20150801")

Filtered File Dates: ['data.20150701_1200', 'data.20150702_1200', 'data.20150706_1200', 'data.20150707_1200', 'data.20150708_1200', 'data.20150709_1200', 'data.20150710_1200', 'data.20150713_1200', 'data.20150714_1200', 'data.20150715_1200', 'data.20150716_1200', 'data.20150717_1200', 'data.20150720_1200', 'data.20150721_1200', 'data.20150722_1200', 'data.20150723_1200', 'data.20150724_1200', 'data.20150727_1200', 'data.20150728_1200', 'data.20150729_1200', 'data.20150730_1200', 'data.20150731_1200']



In [235]:
importlib.reload(test_helper)
ols_model = test_helper.Regression(data_path = FILE_PATH)
ols_model.train(FEATURE_COL_NAMES, list_of_interacting_terms, dataframe = train_df)
ols_model.get_metric(dataframes = test_dfs)

You're using: OLS
Available hyperparams: {'fit_intercept': True, 'copy_X': True, 'n_jobs': None, 'positive': False}
Features being used: ['rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0', "('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')"]
response_corr = 0.02878764732263456
mean_return = 0.00020496155663289543
scale factor = 0.7251932290867417


In [237]:
lasso_model = test_helper.Regression(regression_type="Lasso", data_path=FILE_PATH )
lasso_model.train(FEATURE_COL_NAMES, list_of_interacting_terms, dataframe = train_df)
lasso_model.get_metric(dataframes = test_dfs)

You're using: LASSO
Available hyperparams: {'eps': 0.001, 'n_alphas': 100, 'alphas': None, 'fit_intercept': True, 'precompute': 'auto', 'max_iter': 1000, 'tol': 0.0001, 'copy_X': True, 'cv': None, 'verbose': False, 'n_jobs': None, 'positive': False, 'random_state': None, 'selection': 'cyclic'}
Features being used: ['rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0', "('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')"]
response_corr = 0.02999197590687863
mean_return = 0.00022296778636828244
scale factor = 0.8806622869201416


In [238]:
xgb_model = test_helper.Regression(regression_type="XGBoost", data_path=FILE_PATH)
xgb_model.train(FEATURE_COL_NAMES, list_of_interacting_terms, dataframe = train_df)
xgb_model.get_metric(dataframes = test_dfs)

You're using: XGBOOST
Available hyperparams: {'n_estimators': None, 'objective': 'reg:squarederror', 'max_depth': None, 'max_leaves': None, 'max_bin': None, 'grow_policy': None, 'learning_rate': None, 'verbosity': None, 'booster': None, 'tree_method': None, 'gamma': None, 'min_child_weight': None, 'max_delta_step': None, 'subsample': None, 'sampling_method': None, 'colsample_bytree': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'reg_alpha': None, 'reg_lambda': None, 'scale_pos_weight': None, 'base_score': None, 'missing': nan, 'num_parallel_tree': None, 'random_state': None, 'n_jobs': None, 'monotone_constraints': None, 'interaction_constraints': None, 'importance_type': None, 'device': None, 'validate_parameters': None, 'enable_categorical': False, 'feature_types': None, 'max_cat_to_onehot': None, 'max_cat_threshold': None, 'multi_strategy': None, 'eval_metric': None, 'early_stopping_rounds': None, 'callbacks': None}
Features being used: ['rrirpnxm_nt_0', 'rrirpnxm_lst15